# Object Detection using the YOLO V4 custom-trained model

*by Georgios K. Ouzounis, June 10th, 2021*

In this exercise we will experiment with object detection in still images using the YOLO V4 custom trained model for detecting face masks. Note  that  this is not a face detector. 

## Setup

In [ ]:
# import the relevant libraries
import numpy as np
import cv2 # openCV

In [ ]:
# check the opencv version
print(cv2.__version__)

In [ ]:
# if the openCV version is < 4.4.0 update to the latest otherwise skip this step
!pip install opencv-python==4.5.2.52

## Get the model

mount your Google Drive!

View the various files backed up during the custom model configuration

In [ ]:
%ls /content/drive/MyDrive/my_model/

View the weights files backed up during the custom model training

In [ ]:
%ls /content/drive/MyDrive/my_model/backup/

View the training process chart

In [ ]:
# import the cv2_imshow as a replacement of cv2.imshow to prevent errors
from google.colab.patches import cv2_imshow
chart = cv2.imread("/content/drive/MyDrive/my_model/chart_yolov4-23.png")
cv2_imshow(chart)

In [ ]:
# first create a directory to store the model
%mkdir model

In [ ]:
# enter the directory and download the necessary files 
%cd model
%cp /content/drive/MyDrive/my_model/backup/yolov4_best.weights .
%cp /content/drive/MyDrive/my_model/yolov4.cfg .
%cp /content/drive/MyDrive/my_model/face_mask_classes.names .
%cd ..

##  Get the test data

In [ ]:
# create a directory for test data
%mkdir test_data

In [ ]:
# copy a test file locally or use it directly from the mounted drive 
# customize this example for your drive structure
%cp /content/drive/MyDrive/object_detection/data/test_gko_rs.jpg test_data/
%cp /content/drive/MyDrive/object_detection/data/test_gko_rs_ground_truth.txt test_data/

alternatively access the two files from the author's github repo

In [ ]:
!wget https://github.com/georgiosouzounis/object-detection-yolov4/raw/main/data/custom/test_gko_rs.jpg -O test_data/
!wget https://raw.githubusercontent.com/georgiosouzounis/object-detection-yolov4/main/data/custom/test_gko_rs_ground_truth.txt -O test_data/

## Read test image

In [ ]:
# read file
test_img = cv2.imread('test_data/test_gko_rs.jpg')

In [ ]:
# display test image
cv2_imshow(test_img)

## Image2Blob

convert the image to blob for model compatibility using the opencv builtin  dnn.blobFromImage() method

Argument explanations:
- scalefactor: multiplication factor for each pixel to rescale its intensity in  the range of [0,1]. No contrast stretching is performed. It is set to 1/255.0 = 0.003922.
- new_size: rescaling size for network compatibility. We use (416, 416). Other supported sizes are (320, 320) and (609, 609). The greater the size is the better the prediction accuracy will be but at the cost of higher computational load.
- swapRB: binary flag that if set instructs opencv to swap the red and blue channels. That is because opencv stores images in a BGR order but YOLO requires them in RGB.
- crop: binary flag that if set instructs opencv to crop the image after resizing.


In [ ]:
scalefactor = 1.0/255.0
new_size = (416, 416)
blob = cv2.dnn.blobFromImage(test_img, scalefactor, new_size, swapRB=True, crop=False)

## Customize the YOLO detector

class labels:

In [ ]:
class_labels_path = "/content/model/face_mask_classes.names"
class_labels = open(class_labels_path).read().strip().split("\n")
class_labels

bounding box color definitions: two options

In [ ]:
# declare repeating bounding box colors for each class 
# 1st: create a list colors as an RGB string array
# Example: Red, Green,
class_colors = ["255,0,0","0,255,0"]

#2nd: split the array on comma-seperated strings and for change each string type to integer
class_colors = [np.array(every_color.split(",")).astype("int") for every_color in class_colors]

#3d: convert the array or arrays to a numpy array
class_colors = np.array(class_colors)

## Load and run the model

In [ ]:
# Load the pre-trained model 
yolo_model = cv2.dnn.readNetFromDarknet('model/yolov4.cfg','model/yolov4_best.weights')

In [ ]:
# Read the network layers/components
model_layers = yolo_model.getLayerNames()

In [ ]:
# Extract the output layers
output_layers = [model_layers[model_layer[0] - 1] for model_layer in yolo_model.getUnconnectedOutLayers()]

In [ ]:
# input pre-processed blob into the model
yolo_model.setInput(blob)

# compute the forward pass for the input, storing the results per output layer in a list
obj_detections_in_layers = yolo_model.forward(output_layers)

# verify the number of sets of detections
print("number of sets of detections: " + str(len(obj_detections_in_layers)))

## Analyze the results

The objective now is to get each object detection from each output layer and evaluate  the algorithm's cofidence score against a threshold. For high confidence detections we extract the class ID and the bounding box info and apply non-maximum suppression.

In [ ]:
%cp /content/drive/MyDrive/object_detection/object_detection_functions.py .

In [ ]:
from object_detection_functions import object_detection_analysis_with_nms

In [ ]:
score_threshold = 0.5
nms_threshold = 0.4
result, winner_boxes = object_detection_analysis_with_nms(test_img, class_labels, class_colors, obj_detections_in_layers, score_threshold, nms_threshold)

In [ ]:
cv2_imshow(result)

create a 2D list containing the bounding box end points for each detection.

create a 2D list containing the ground truth box end points for each object.

both lists must be of the same dimensions

In [ ]:
import io

ground_truth_boxes = []

with io.open("test_data/test_gko_rs_ground_truth.txt", mode="r", encoding="utf-8") as f:
  for line in f:
    ground_truth_boxes.append(line.split())

for i in range(0, len(ground_truth_boxes)):
  for j in range(0, len(ground_truth_boxes[i])):
    ground_truth_boxes[i][j] = int(ground_truth_boxes[i][j])

### Compute the IoU metric

The green bounding boxes depict the detected object while the red, the ground truth object

In [ ]:
from object_detection_functions import object_detection_iou

In [ ]:
# create a copy of the test image
iou_image = test_img.copy()

# print the ground truth and detection bounding boxes, and the IoU value
iou_image, iou_value = object_detection_iou(test_img, winner_boxes[0], ground_truth_boxes[0])

In [ ]:
cv2_imshow(iou_image)